In [1]:
import onnxruntime as ort
import cv2
import numpy as np
import time 

def read_img(source_path:str, source_type:str):
    if source_type == "image":
        return cv2.imread(source_path)

def preprocessing(img, input_size:tuple =(640, 640)):
    
    if img.shape[0] != input_size[0] or img.shape[1] != input_size[1]:
        input_img_resized = cv2.resize(img, input_size)
    
    input_img_rgb = cv2.cvtColor(input_img_resized, cv2.COLOR_BGR2RGB)

    input_img_normalized = input_img_rgb / 255.0

    input_tensor = input_img_normalized.transpose(2, 0, 1).astype(np.float32)
    input_tensor = np.expand_dims(input_tensor, axis=0)

    return input_tensor

def load_model(model_path:str):
    return ort.InferenceSession(model_path)

def post_process(outputs):
    # Example post-processing code, adjust according to your model's output format
    boxes, scores, class_ids = outputs  # Split the output tensors if necessary

    # Thresholding and NMS to remove unnecessary boxes
    threshold = 0.5
    nms_threshold = 0.4

    indices = cv2.dnn.NMSBoxes(boxes, scores, threshold, nms_threshold)

    # Extract the detected objects
    detected_objects = []
    for i in indices:
        box = boxes[i[0]]
        score = scores[i[0]]
        class_id = class_ids[i[0]]
        detected_objects.append((box, score, class_id))
    
    return detected_objects

def visulize_objects(input_image, detected_objects):
    # Draw the bounding boxes on the image
    for box, score, class_id in detected_objects:
        x, y, w, h = box
        cv2.rectangle(input_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        label = f'{class_id}: {score:.2f}'
        cv2.putText(input_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the image with bounding boxes
    cv2.imshow('Detected Objects', input_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [38]:


source_path = r"C:\Users\p.alishah\Desktop\Projects\python\models\ultralytics\data\test\val_all_data-56-_jpg.rf.fd1cd988f8460e5e0f2f51399df2c348.jpg"
model_path = r"C:\Users\p.alishah\Desktop\Projects\python\models\ultralytics\runs\detect\train8\weights\best.onnx"

input_img = read_img(source_path, "image")
preprocessed_img = preprocessing(input_img.copy())

ort_session = load_model(model_path)

start_time = time.time()
outputs = ort_session.run(None, {'images': preprocessed_img})
end_time = time.time()

print("Latency:", (end_time - start_time))
print("FPS:", 1/(end_time - start_time))

Latency: 0.1678469181060791
FPS: 5.957809719020285


In [ ]:
def postprocess(input_image, output):
    """
    Performs post-processing on the model's output to extract bounding boxes, scores, and class IDs.

    Args:
        input_image (numpy.ndarray): The input image.
        output (numpy.ndarray): The output of the model.

    Returns:
        numpy.ndarray: The input image with detections drawn on it.
    """

    # Transpose and squeeze the output to match the expected shape
    outputs = np.transpose(np.squeeze(output[0]))

    # Get the number of rows in the outputs array
    rows = outputs.shape[0]

    # Lists to store the bounding boxes, scores, and class IDs of the detections
    boxes = []
    scores = []
    class_ids = []

    # Calculate the scaling factors for the bounding box coordinates
    x_factor = self.img_width / self.input_width
    y_factor = self.img_height / self.input_height

    # Iterate over each row in the outputs array
    for i in range(rows):
        # Extract the class scores from the current row
        classes_scores = outputs[i][4:]

        # Find the maximum score among the class scores
        max_score = np.amax(classes_scores)

        # If the maximum score is above the confidence threshold
        if max_score >= self.confidence_thres:
            # Get the class ID with the highest score
            class_id = np.argmax(classes_scores)

            # Extract the bounding box coordinates from the current row
            x, y, w, h = outputs[i][0], outputs[i][1], outputs[i][2], outputs[i][3]

            # Calculate the scaled coordinates of the bounding box
            left = int((x - w / 2) * x_factor)
            top = int((y - h / 2) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)

            # Add the class ID, score, and box coordinates to the respective lists
            class_ids.append(class_id)
            scores.append(max_score)
            boxes.append([left, top, width, height])

    # Apply non-maximum suppression to filter out overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, scores, self.confidence_thres, self.iou_thres)

    # Iterate over the selected indices after non-maximum suppression
    for i in indices:
        # Get the box, score, and class ID corresponding to the index
        box = boxes[i]
        score = scores[i]
        class_id = class_ids[i]

        # Draw the detection on the input image
        self.draw_detections(input_image, box, score, class_id)

    # Return the modified input image
    return input_image